In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Multi Agent Building

Dependency injection shown in [02.tool_calling_and_dependency_injection.ipynb](./02.tool_calling_and_dependency_injection.ipynb) allow us to build multi agent with intuitive coding. `Agent` class has the argument `subagents` that accepts `list[Agent]` typed variable, and sub-agents are dynamically converted to tools which invokes agent using dependency injection feature. This notebook shows examples of multi-agent building. 

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila import Agent, InMemoryConversationMemory
from langrila.anthropic import AnthropicClient
from langrila.google import GoogleClient
from langrila.openai import OpenAIClient

# Instantiate client

In [4]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(api_key_env_name="GEMINI_API_KEY")

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(api_key_env_name="ANTHROPIC_API_KEY")

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

# Define tools

Using dummy tools. As mentioned in [02.tool_calling_and_dependency_injection.ipynb](./02.tool_calling_and_dependency_injection.ipynb), tool can have the pydantic model argument.

In [5]:
import random
from enum import Enum

from pydantic import BaseModel, Field


class MusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class EqualizerSettings(BaseModel):
    hz_100: int = Field(0, ge=-12, le=12, description="The 100 Hz band in dB.")
    hz_200: int = Field(0, ge=-12, le=12, description="The 200 Hz band in dB.")
    hz_400: int = Field(0, ge=-12, le=12, description="The 400 Hz band in dB.")
    hz_800: int = Field(0, ge=-12, le=12, description="The 800 Hz band in dB.")
    hz_1600: int = Field(0, ge=-12, le=12, description="The 1600 Hz band in dB.")
    hz_3200: int = Field(0, ge=-12, le=12, description="The 3200 Hz band in dB.")
    hz_6400: int = Field(0, ge=-12, le=12, description="The 4000 Hz band in dB.")


def start_music(genre: MusicGenre) -> str:
    """
    Start playing a random music with the given the genre.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to play.

    Returns
    ----------
    str
        A message indicating that the music has started.
    """

    bpms = {
        MusicGenre.rock: 150,
        MusicGenre.pop: 110,
        MusicGenre.jazz: 120,
        MusicGenre.classical: 76,
        MusicGenre.hip_hop: 100,
    }

    return (
        f"Starting music! Genre: {genre}, "
        f"Volume: {random.uniform(0.0, 1.0)}, "
        f"BPM: {bpms[genre]}"
    )


def change_music_volume(volume: float) -> str:
    """
    Change the music volume.

    Parameters
    ----------
    volume : float
        The new volume to set. It should be between 0.0 and 1.0.

    Returns
    ----------
    str
        A message indicating that the volume has been changed.
    """
    return f"Changing volume to {volume}"


def change_music(genre: MusicGenre) -> str:
    """
    Change the music.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to change to.

    Returns
    ----------
    str
        A message indicating that the music has been changed.
    """
    bpm_ranges = {
        MusicGenre.rock: (120, 180),
        MusicGenre.pop: (80, 140),
        MusicGenre.jazz: (70, 120),
        MusicGenre.classical: (70, 100),
        MusicGenre.hip_hop: (80, 120),
    }

    new_bpm = random.randint(bpm_ranges[genre][0], bpm_ranges[genre][1])

    return f"Turning to another music! Genre: {genre}, BPM: {new_bpm}"


def change_equalizer_settings(settings: EqualizerSettings) -> str:
    """
    Change the equalizer settings of the music.

    Parameters
    ----------
    settings : EqualizerSettings
        The new equalizer settings to set.

    Returns
    ----------
    str
        A message indicating that the equalizer settings have been changed.
    """
    return f"Changing equalizer settings to {settings.model_dump(exclude_none=True, exclude_unset=True)}"


def turn_light_on() -> str:
    """
    Turn the lights on.

    Returns
    ----------
    str
        A message indicating that the lights are turning on.
    """
    brightness = random.uniform(0.5, 1.0)
    return "Lights are now on! Brightness: {:.2f}".format(brightness)


def dim_lights(brightness: float) -> bool:
    """
    Dim the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully dimmed.
    """
    return f"Lights are now set to {brightness}"


def power_disco_ball(power: bool) -> bool:
    """
    Powers the spinning dissco ball on or off.

    Parameters
    ----------
    power : bool
        Whether to power the disco ball or not.

    Returns
    ----------
    bool
        Whether the disco ball is spinning or not.
    """
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def adjust_lights(brightness: float) -> bool:
    """
    Adjust the brightness of the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully brightened.
    """
    return f"Lights are now set to {brightness}"

# Multi-Agent

`Agent` class accepts `subagents` argument which generates dynamically tools to run agent. Langrila supports multi agent with multi client. In langrila, we can build orchestrator-typed multi-agent, not graph-based multi-agent. The orchestrator routes the execution of tools to task-specific agents, aggregates the results, and outputs the final answer.

If you don't specify the conversation memory for sub-agent, `InMemoryConversationMemory` is internally used in default, which is not persisted automatically within a sub-agent. Each sub-agentalways need own conversation memory because the state within the multi-agent is kept or updated based on the conversation history and response schema if specified. Please be aware of using internal memory even if you don't specify the conversation memory for the sub-agent.

However, agent has `use_last_n_turns` argument to control how many conversation history we consider, default value is -1 which means all conversation history within the agent is used. The `turn` in this place is the sequence from the user input to the final answer of the agent including internal looped conversation in the agent. If 0, no conversation history is used. Even you specified `use_last_n_turns`, entire conversation memory is stored for debugging, not overwritten. `use_last_n_turns` parameter is used to extract the specified turn history from the entire conversation history.

## Task specific agent. 

In [6]:
lights_agent = Agent(
    client=vertexai_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=[turn_light_on, adjust_lights],
    use_last_n_turns=0,
)

disco_ball_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    tools=[power_disco_ball],
    max_tokens=500,
    use_last_n_turns=0,
)

music_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[
        start_music,
        change_equalizer_settings,
        change_music,
        change_music_volume,
    ],
    use_last_n_turns=0,
)

## Orchestrator agent.

The thing you have to do for building multi-agent is to pass agents to the orchestrator agent. Planning mode is supported for multi agent.

In [7]:
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    planning=True,
)

## Invoke multi-agent

In [8]:
prompt = "Turn this place into a party mood! Then tell me the details of the settings such as equalizer setting"

In [9]:
response = orchestrator.generate_text(prompt=prompt)

[2025-01-09 23:45:09] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood! Then tell me the details of the settings such as equalizer setting\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - start_music: Start playing a random music with the given the genre.\n  - change_equalizer_settings: Change the equalizer settings of the music.\n  - change_music: Change the music.\n  - change_music_volume: Change the music volume.\n')]
[2025-01-09 23:45:09] INFO | root: Generating text
[2025-0

As you can see in the log message above, the orchestrator agent assigned subdivided task to each sub-agents and finally generated answer.

In [10]:
print(response.contents[0].text)

### Party Mood Settings Activated!

1. **Lighting**: 
   - The lights have been turned on and set to a brightness level of **75%**.

2. **Disco Ball**: 
   - The disco ball is now powered on and spinning!

3. **Music**: 
   - Random **pop music** is playing at a lively volume level of **0.7** (70%).

4. **Equalizer Settings**: 
   - The equalizer has been adjusted to enhance the music experience by boosting the **bass and treble**. The overall music volume has been increased to **80%**.

Enjoy the party atmosphere! If you need anything else, just let me know!


Usage is collected all over the agent including sub-agents.

In [11]:
list(response.usage.items())

[('lights_agent',
  Usage(model_name='gemini-2.0-flash-exp', prompt_tokens=159, output_tokens=28)),
 ('music_agent',
  Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=1083, output_tokens=130)),
 ('root',
  Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=1937, output_tokens=518)),
 ('disco_ball_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=892, output_tokens=150))]

Top-level orchestrator name is `root`.

Individual sub-agents store their own inputs and outputs. You can access those conversation histories by calling `load_history()` of focused agent. Here is an example to access to the history of music_agent.

In [12]:
orchestrator.subagents[2]

Agent(name=music_agent)

In [13]:
orchestrator.subagents[2].load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Start playing random music in a lively genre and set the volume to a suitable level.')], name=None),
  Response(type='Response', role='assistant', contents=[ToolCallResponse(name='start_music', args='{"genre": "pop"}', call_id='call_QChDtEVtnW6B5ANWq3yhusB4'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.7}', call_id='call_OmjIRVdjyGR34HXm96cgz3a5')], usage=Usage(model_name=None, prompt_tokens=0, output_tokens=0), raw=None, name='music_agent', is_last_chunk=None, prompt=None),
  Prompt(type='Prompt', role='user', contents=[ToolUsePrompt(output='Starting music! Genre: pop, Volume: 0.9348266627895883, BPM: 110', error=None, call_id='call_QChDtEVtnW6B5ANWq3yhusB4', args='{"genre": "pop"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.7', error=None, call_id='call_OmjIRVdjyGR34HXm96cgz3a5', args='{"volume": 0.7}', name='change_music_volume')], name=None),
  Response(type='Response', role='

# State in the agent

Agent state in existing agent frameworks has some issues on readability, special argument, and traceability. Ideally, state should be expressed by both the dependencies between agents and response schema, no special manner should be taken. It means the state is updated by llm based on the conversation history and response schema while its scope is limited by the agent's dependencies.

In langrila, the combination of the dependencies between the agents and structured output can take the place of the state in agent. Note that planning mode is supported even if multi agent case.

## Response schema

In [14]:
class DiscoBallSchema(BaseModel):
    power: bool = Field(..., description="power state of the disco ball.")


class MusicPropertySchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music.",
    )
    volume: float = Field(
        ...,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    bpm: int = Field(
        ...,
        description="The beats per minute of the music.",
    )


class ChangedMusicSchema(BaseModel):
    genre: MusicGenre = Field(
        None,
        description="The genre of music.",
    )
    volume: float = Field(
        None,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    bpm: int = Field(
        None,
        description="The beats per minute of the music.",
        ge=50,
        le=200,
    )


class MusicSchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music.",
    )
    volume: float = Field(
        ...,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    bpm: int = Field(
        ...,
        description="The beats per minute of the music.",
    )
    equalizer_settings: EqualizerSettings = Field(
        ...,
        description="The equalizer settings.",
    )


class LightsSchema(BaseModel):
    brightness: float = Field(
        ...,
        description="The brightness level to set the lights to.",
        ge=0,
        le=1,
    )


class ResponseSchema(BaseModel):
    disco_ball: DiscoBallSchema = Field(..., description="The disco ball settings.")
    music: MusicSchema = Field(..., description="The music settings.")
    lights: LightsSchema = Field(..., description="The lights settings.")

## Task specific agent.

To keep the state in each agent, `use_last_n_turns` parameter should be greater than or equal to 1, otherwise -1 to use the entire conversation history. 

In [15]:
from langrila import SystemPrompt

lights_agent = Agent(
    client=google_dev_client,
    model="gemini-1.5-flash-002",
    temperature=0.0,
    tools=[turn_light_on, adjust_lights],
    response_schema_as_tool=LightsSchema,
    use_last_n_turns=1,  # Use the last turn for the state of the lights.
)

disco_ball_agent = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    tools=[power_disco_ball],
    max_tokens=500,
    response_schema_as_tool=DiscoBallSchema,
    use_last_n_turns=1,  # Use the last turn for the state of the disco ball.
)

music_control_agent = Agent(
    client=google_dev_client,
    model="gemini-1.5-flash-002",
    temperature=0.0,
    tools=[start_music, change_music, change_music_volume],
    planning=True,
    response_schema_as_tool=MusicPropertySchema,
    use_last_n_turns=1,  # Use the last turn for the state of the music control.
    system_instruction=SystemPrompt(
        contents="You can start the music, or change the music genre and volume."
    ),
)

equalizer_control_agent = Agent(
    client=google_dev_client,
    model="gemini-1.5-flash-002",
    temperature=0.0,
    tools=[change_equalizer_settings],
    planning=True,
    response_schema_as_tool=EqualizerSettings,
    use_last_n_turns=1,  # Use the last turn for the state of the music control.
    system_instruction=SystemPrompt(contents="You can change the equalizer settings."),
)

## Orchestrator agent

In [16]:
# Orchestrator agent as a sub-agent
music_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    subagents=[music_control_agent, equalizer_control_agent],
    max_tokens=500,
    response_schema_as_tool=MusicSchema,
    use_last_n_turns=1,  # Use the last turn for the state of the music.
    system_instruction=SystemPrompt(
        contents="You can start the music, change the genre, volume, and equalizer settings."
    ),
)

# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    conversation_memory=InMemoryConversationMemory(),
    response_schema_as_tool=ResponseSchema,  # as needed
    planning=True,
    use_last_n_turns=1,
    system_instruction=SystemPrompt(contents="You can control the lights, disco ball, and music."),
)

## Invoke agent

In [17]:
prompt = "Turn this place into a lively party mood!"

In [18]:
response = orchestrator.generate_text(prompt=prompt)

[2025-01-09 23:45:32] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-09 23:45:32] INFO | root: Generating text
[2025

In [19]:
list(response.usage.items())

[('music_control_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=2048, output_tokens=170)),
 ('root',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=2679, output_tokens=418)),
 ('lights_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=790, output_tokens=35)),
 ('music_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=4062, output_tokens=655)),
 ('equalizer_control_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=4643, output_tokens=231)),
 ('disco_ball_agent',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=590, output_tokens=53))]

In [20]:
print(response.contents[0].text)

{"disco_ball": {"power": true}, "music": {"genre": "pop", "volume": 0.8, "bpm": 120, "equalizer_settings": {"hz_100": 6, "hz_200": 6, "hz_400": 0, "hz_800": 0, "hz_1600": 0, "hz_3200": 4, "hz_6400": 4}}, "lights": {"brightness": 0.8}}


Validation response schema

In [21]:
valid_resposne = ResponseSchema.model_validate_json(response.contents[0].text)
valid_resposne.model_dump()

{'disco_ball': {'power': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>,
  'volume': 0.8,
  'bpm': 120,
  'equalizer_settings': {'hz_100': 6,
   'hz_200': 6,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 4,
   'hz_6400': 4}},
 'lights': {'brightness': 0.8}}

Top-level orchestrator and each sub-agent has own conversation memory.

In [22]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a

Conversation memory of sub-agent

In [23]:
orchestrator.subagents[2]

Agent(name=music_agent)

In [24]:
orchestrator.subagents[2].load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Start playing upbeat pop music for a lively party mood. Set the volume to a lively level and enhance the bass and treble using the equalizer.')], name=None),
  Response(type='Response', role='assistant', contents=[TextResponse(text="Certainly! I'd be happy to help you set up some upbeat pop music for a lively party mood with enhanced bass and treble. Let's break this down into steps and use the available tools to achieve what you're looking for.\n\nFirst, let's start the music and set the volume. Then, we'll adjust the equalizer settings to enhance the bass and treble. Here's how we'll do it:\n\n1. Start playing pop music and set the volume\n2. Adjust the equalizer settings\n\nLet's begin with starting the music and setting the volume:"), ToolCallResponse(name='route_music_control_agent', args='{"instruction": "Start playing upbeat pop music and set the volume to a lively level. For a party mood, we should set the volume t

In [25]:
orchestrator.subagents[2].subagents[0]

Agent(name=music_control_agent)

In [26]:
orchestrator.subagents[2].subagents[0].load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nStart playing upbeat pop music and set the volume to a lively level. For a party mood, we should set the volume to around 0.8 (80% of maximum volume).\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music: Change the music.\n- change_music_volume: Change the music volume.\n')], name=None),
  Response(type='Response', role='assistant', contents=[TextResponse(text='Plan:\n\n1. **Use `start_music`:** Invoke `start_music` with the genre "pop".\n2. **Use `change_music_volume`:** Invoke `change_music_volume` with the volume level 0.8.')], usage=Usage(model_name=None, prompt_

Those final answers within each agent are regarded as the state in the agent. If necessary, also you can inject the custom context into tools as mentioned in [./02.tool_calling_and_dependency_injection.ipynb](./02.tool_calling_and_dependency_injection.ipynb), so please select more preferable way.

Next turn prompt

In [27]:
prompt = "Next, please make a calm atmosphere with a jazz music."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-09 23:46:04] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nNext, please make a calm atmosphere with a jazz music.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-09 23:46:04] INFO | root: Generati

In [28]:
list(response.usage.items())

[('music_control_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=4459, output_tokens=247)),
 ('root',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=8410, output_tokens=520)),
 ('lights_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=1334, output_tokens=21)),
 ('music_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=6844, output_tokens=673)),
 ('equalizer_control_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=6561, output_tokens=220)),
 ('disco_ball_agent',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=1082, output_tokens=50))]

Validation response schema

In [29]:
valid_resposne = ResponseSchema.model_validate_json(response.contents[0].text)
valid_resposne.model_dump()

{'disco_ball': {'power': False},
 'music': {'genre': <MusicGenre.jazz: 'jazz'>,
  'volume': 0.3,
  'bpm': 72,
  'equalizer_settings': {'hz_100': 2,
   'hz_200': 0,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 1,
   'hz_6400': 0}},
 'lights': {'brightness': 0.2}}

Conversation memory in top-level orchestrator.

In [30]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a

Conversation memory of sub-agent.

In [31]:
orchestrator.subagents[2]

Agent(name=music_agent)

In [32]:
orchestrator.subagents[2].load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Start playing upbeat pop music for a lively party mood. Set the volume to a lively level and enhance the bass and treble using the equalizer.')], name=None),
  Response(type='Response', role='assistant', contents=[TextResponse(text="Certainly! I'd be happy to help you set up some upbeat pop music for a lively party mood with enhanced bass and treble. Let's break this down into steps and use the available tools to achieve what you're looking for.\n\nFirst, let's start the music and set the volume. Then, we'll adjust the equalizer settings to enhance the bass and treble. Here's how we'll do it:\n\n1. Start playing pop music and set the volume\n2. Adjust the equalizer settings\n\nLet's begin with starting the music and setting the volume:"), ToolCallResponse(name='route_music_control_agent', args='{"instruction": "Start playing upbeat pop music and set the volume to a lively level. For a party mood, we should set the volume t

Now we keep only the last turn conversation, so the agent and sub-agents will answer based on the last state if the user ask the agent to change a specific setting. For example, when we ask to change the music to another one, the agent update a specific state related to the user query, using the last state for other information.

In [33]:
prompt = "Please change to another music of the same genre."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-09 23:46:41] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlease change to another music of the same genre.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-09 23:46:41] INFO | root: Generating te

In [34]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': False},
 'music': {'genre': <MusicGenre.jazz: 'jazz'>,
  'volume': 0.3,
  'bpm': 105,
  'equalizer_settings': {'hz_100': 2,
   'hz_200': 0,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 1,
   'hz_6400': 0}},
 'lights': {'brightness': 0.2}}

In [35]:
list(response.usage.items())

[('music_control_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=4735, output_tokens=220)),
 ('root',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=6371, output_tokens=215)),
 ('music_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=4266, output_tokens=315))]

In [36]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a

`use_last_n_turns` extracts the specified turns history from the entire history above. Here is an internal process.

In [38]:
entire_history = orchestrator.load_history()
used_history = orchestrator._pick_last_n_turns_history(entire_history)
used_history

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlease change to another music of the same genre.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response',

In [41]:
used_history[0] == entire_history[-1]

True